In [1]:
import numpy as np
import mujoco
import mediapy as media
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import pandas as pd 
import cv2

In [2]:
xml = './low_cost_robot/scene.xml'
model = mujoco.MjModel.from_xml_path(xml)

model.vis.global_.offheight = 480
model.vis.global_.offwidth = 640

jname = 'joint6'

renderer = mujoco.Renderer(model, height=480, width=640)

camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 1


In [72]:
# Function to load .pkl file
def load_keypoints(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data 

# Function to save 3D keypoints plot as an image
def keypoint_joint(keypoints_3d_frame):
    # Inverse Whole body
    # joint_0 = keypoints_3d_frame[0:5]  # Head
    # joint_1 = keypoints_3d_frame[5:7]  # Shoulder
    # joint_2 = keypoints_3d_frame[7:11]  # Hands
    # joint_3 = keypoints_3d_frame[11:13] # Pelvis
    # joint_4 = keypoints_3d_frame[13:15] # Knee
    # joint_5 = keypoints_3d_frame[15:17] # Foot
    # Whole body
    # joint_0 = keypoints_3d_frame[15:17]  # Foot
    # joint_1 = keypoints_3d_frame[13:15]  # Knee
    # joint_2 = keypoints_3d_frame[11:13]  # Pelvis
    # joint_3 = keypoints_3d_frame[7:11] # Hands
    # joint_4 = keypoints_3d_frame[5:7] # Shoulder
    # joint_5 = keypoints_3d_frame[0:5] # Head
    # Custom_1
    # joint_0 = keypoints_3d_frame[15:17]  # Foot
    # joint_1 = keypoints_3d_frame[13:15]  # Knee
    # joint_2 = keypoints_3d_frame[11:13]  # Pelvis
    # joint_3 = keypoints_3d_frame[5:7] # Shoulder
    # joint_4 = keypoints_3d_frame[0:5] # Head
    # joint_5 = keypoints_3d_frame[7:11] # Hands
    # Edge Model_custom_1
    # joint_0 = keypoints_3d_frame[[8,11,7,10]]  # Foot
    # joint_1 = keypoints_3d_frame[[5,4]]  # Knee
    # joint_2 = keypoints_3d_frame[[0,1,2]]  # Pelvis
    # joint_3 = keypoints_3d_frame[[14,13,9,6,3]] # Shoulder
    # joint_4 = keypoints_3d_frame[[12,15]] # Head
    # joint_5 = keypoints_3d_frame[[17,19,21,23,16,18,20,22]] # Hands
    # Edge Model_custom_2 
    # joint_0 = keypoints_3d_frame[[8,11,7,10]]  # Foot
    # joint_1 = keypoints_3d_frame[[5,4]]  # Knee
    # joint_2 = keypoints_3d_frame[[0,1,2]]  # Pelvis
    # joint_3 = keypoints_3d_frame[[17,19,21,23,16,18,20,22]] # Hands
    # joint_4 = keypoints_3d_frame[[14,13,9,6,3]] # shoulders
    # joint_5 = keypoints_3d_frame[[12,15]] # Head
    # Edge Model_custom_3 
    joint_0 = keypoints_3d_frame[[11, 8]]  # Foot
    joint_1 = keypoints_3d_frame[[5, 2]]  # Knee
    joint_2 = keypoints_3d_frame[[0, 3]]  # Pelvis
    joint_3 = keypoints_3d_frame[[6, 9]] # Hands
    joint_4 = keypoints_3d_frame[[12]] # shoulders
    joint_5 = keypoints_3d_frame[[15]] # Head
    
    return joint_0, joint_1, joint_2, joint_3, joint_4, joint_5

def frobenius_norm(w):
    return np.sqrt(np.sum(w**2))

def distance_norm(w):
    return np.sum(np.sqrt(np.sum(w**2, axis=1)))

def map_to_closest(values, target_values):
    return np.array([[min(target_values, key=lambda x: abs(x - v)) for v in row] for row in values])

In [73]:
# Load keypoints from .pkl file
file_path = './test_We_Will_Rock_You.pkl'  # Replace with your .pkl file path

In [76]:
data = load_keypoints(file_path)

results = []

# Extract and process 3D keypoints
full_pose = data.get('full_pose')  # Frame-by-frame 3D keypoints, shape (N, 17, 3)
if full_pose is not None:
    for frame_idx in range(full_pose.shape[0]):  # Iterate over frames
        keypoints_3d_frame = full_pose[frame_idx]
        joints = keypoint_joint(keypoints_3d_frame)
        distance_norms = [frobenius_norm(joint) for joint in joints]
        distance_norms = np.array(distance_norms)
        results.append(distance_norms)
else:
    print("No 3D keypoints found.")

variances = [result - results[0] for result in results]

max_values = np.max(np.abs(variances), axis=0)

normalized_variances = variances / max_values

# step = 0.05
# target_values = np.arange(-1, 1 + step, step)
# normalized_variances = map_to_closest(normalized_variances, target_values)


ranges = [
    [np.pi/2, -np.pi/2],  # Column 1 range
    [np.pi/8, -np.pi/8],  # Column 2 range
    [np.pi/2, np.pi/4],  # Column 3 range
    [np.pi/2, -np.pi/2],  # Column 4 range
    [np.pi/2, -np.pi/2],  # Column 5 range
    [np.pi/4,        0]          # Column 6 range
]

degree_normalize_variance = np.zeros_like(normalized_variances)

for col in range(normalized_variances.shape[1]):
    if col == 2 or col == 5:
        min_val, max_val = ranges[col]
        degree_normalize_variance[:, col] = (normalized_variances[:, col] + 1) * (max_val + min_val) / 2
    else:
        min_val, max_val = ranges[col]
        degree_normalize_variance[:, col] = normalized_variances[:, col] * min_val

fps = 120
total_frames = 4
data = mujoco.MjData(model)

frames = []
for i in range(normalized_variances.shape[0]-1):
    previous = degree_normalize_variance[i]
    current = degree_normalize_variance[i+1]

    data.qpos[0] = previous[0]
    data.qpos[1] = previous[1]
    data.qpos[2] = previous[2]
    data.qpos[3] = previous[3]
    data.qpos[4] = previous[4]
    data.qpos[5] = previous[5]
    previous = data.qpos.copy()

    data.qpos[0] = current[0]
    data.qpos[1] = current[1]
    data.qpos[2] = current[2]
    data.qpos[3] = current[3]
    data.qpos[4] = current[4]
    data.qpos[5] = current[5]
    current = data.qpos.copy()

    interp = np.linspace(previous, current, total_frames)

    frame_sequence = []
    # Generate frames
    for i in range(total_frames):
        # Update the robot's joint positions
        data.qpos[:len(previous)] = interp[i]
        mujoco.mj_forward(model, data)

        # Render the scene
        renderer.update_scene(data, camera)
        frame = renderer.render()
            
        frame_sequence.append(frame)
    frames.extend(frame_sequence)

media.show_video(frames, fps=fps, loop=False)

This browser does not support the video tag.


In [69]:
height, width, _ = frames[0].shape  # Assuming all frames have the same dimensions
video_path = "output_video.mp4"  # File path for the output video
fps = 120  # Frames per second

# Initialize the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
video_writer = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

# Write each frame
for frame in frames:
    video_writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # Convert RGB to BGR (OpenCV uses BGR format)

# Release the writer
video_writer.release()
print(f"Video saved at {video_path}")

Video saved at output_video.mp4


In [71]:
!ffmpeg -i output_video.mp4 -i test_We_Will_Rock_You.mp4 -c:v copy -map 0:v:0 -map 1:a:0 -shortest We_Will_Rock_You_dance.mp4 -y

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:
import cv2
import numpy as np

def align_videos(video1_path, video2_path, output_path):
    # Open the video files
    cap1 = cv2.VideoCapture(video1_path)
    cap2 = cv2.VideoCapture(video2_path)

    # Get properties of both videos
    fps1 = int(cap1.get(cv2.CAP_PROP_FPS))
    fps2 = int(cap2.get(cv2.CAP_PROP_FPS))

    frame_width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))

    frame_width2 = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height2 = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Use the lower FPS to synchronize frames
    target_fps = min(fps1, fps2)

    # Determine the output frame size
    frame_width = max(frame_width1, frame_width2)
    frame_height = max(frame_height1, frame_height2)

    # Define the codec and create VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    out = cv2.VideoWriter(output_path, fourcc, target_fps, (frame_width * 2, frame_height))

    # Frame skipping counters
    frame_skip1 = fps1 // target_fps
    frame_skip2 = fps2 // target_fps

    frame_count1 = 0
    frame_count2 = 0

    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        # Break the loop if both videos are done
        if not ret1 and not ret2:
            break

        # Skip frames to match the target FPS
        if frame_count1 % frame_skip1 != 0:
            ret1 = False
        if frame_count2 % frame_skip2 != 0:
            ret2 = False

        # Increment frame counters
        frame_count1 += 1
        frame_count2 += 1

        # Handle blank frames when one video ends early
        if not ret1:
            frame1 = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
        else:
            frame1 = cv2.resize(frame1, (frame_width, frame_height))

        if not ret2:
            frame2 = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
        else:
            frame2 = cv2.resize(frame2, (frame_width, frame_height))

        # Concatenate frames side by side
        combined_frame = cv2.hconcat([frame1, frame2])

        # Write the combined frame to the output video
        out.write(combined_frame)

    # Release resources
    cap1.release()
    cap2.release()
    out.release()
    print(f"Aligned video saved at {output_path}")

# Usage example
video1 = "output_video.mp4"
video2 = "test_Twist_And_Shout.mp4"
output = "synchronized_video.mp4"

align_videos(video1, video2, output)


Aligned video saved at synchronized_video.mp4
Aligned video saved at synchronized_video.mp4
